In [159]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
from math import log
from collections import Counter

In [160]:
df_v7 = pd.read_csv('v7_auto_labeled_sort_with_box_office_1997-2017.csv')

In [161]:
df_v7.head()

,n_index,previous_index_num,movie_title_txt_num,imdb_id,n_of_posters,total_box_office,with_adjusted_box_office,total_label,year,movie_title,year_box_office_label,Rnum_year_box_office_label,groups_num,train_or_test,budget
0,10037,5588,5854,tt0120376,2,217,422,0,1997,Trojan War,0,R0,18,test,0
1,10036,5576,5839,tt0118870,3,552,1076,0,1997,Cold Around the Heart,0,R0,17,test,0
2,10035,5548,5800,tt0130297,1,967,1606,0,1997,Sweet Jane,0,R0,16,test,0
3,10034,5467,5708,tt0028905,1,2126,3534,0,1997,Homme Nu L,0,R0,15,test,0
4,10033,5365,5592,tt0119327,2,3620,7068,0,1997,Hugo Pool,0,R0,14,train,0


In [162]:
df_v7_years = df_v7.groupby('year')

In [163]:
df_v7_years.groups.keys()

dict_keys([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [207]:
#check the detail of specified year

def test_label_each_year_to_R0_R9(year):
    #prepared inital paraments
    count = 0
    df_v7_each_year = df_v7_years.get_group(year)
    
    print('year: ', year)
    n_poster_num = df_v7_each_year['n_of_posters'].sum()
    print('number of poster: ', n_poster_num)
    group_num_threadhold = int(n_poster_num/5)
    print('group num threadhold: ', group_num_threadhold)
    print()
    for index in range(0, len(df_v7_each_year)):
        print('index: ', index)
        n_poster_each_index = df_v7_each_year['n_of_posters'].iloc[index]
        print('number of posters in this index: ', n_poster_each_index)
        count = count + n_poster_each_index
        print('count: ', count)
        print()
        #print()

In [208]:
#test_label_each_year_to_R0_R9(1997)

In [209]:
def get_stop_index_each_year_to_R0_R9(year):
    stop_index_list = []
    #prepared inital paraments
    count = 0
    df_v7_each_year = df_v7_years.get_group(year)
    
    print('year: ', year)
    n_poster_num = df_v7_each_year['n_of_posters'].sum()
    #print('number of poster: ', n_poster_num)
    group_num_threadhold = int(n_poster_num/5)
    #print('group num threadhold: ', group_num_threadhold)
    for index in range(0, len(df_v7_each_year)):
        #print('index: ', index)
        n_poster_each_index = df_v7_each_year['n_of_posters'].iloc[index]
        #print('number of posters in this index: ', n_poster_each_index)
        count = count + n_poster_each_index
        if count >= group_num_threadhold: 
            #print('***stop index: ', index)
            if count == group_num_threadhold:
                stop_index_list.append(index)
            #print('count: ', count)
            else:
                #print('nooooo')
                prev_count = count-df_v7_each_year['n_of_posters'].iloc[index]
                #print('previous: ', prev_count)
                #print('over: ', count)
                distant_prev = group_num_threadhold - prev_count
                #print('distant of prev_count: ', distant_prev)
                distant_over = count - group_num_threadhold
                #print('distant of over_count: ', distant_over)
                find_min = [distant_prev, distant_over]
                min_dis = min(find_min)
                #print('who is min: ', min_dis)
                #choose the corresponding index of the minimum number
                choose_index = find_min.index(min_dis)
                #print('choose index: ', choose_index)
                #0=prev, 1=over
                if choose_index == 0:
                    #print('I choose previous: ', prev_count)
                    #print('***stop index will be: ', index-1)
                    stop_index_list.append(index-1)
                else:
                    #print('I choose over: ', count)
                    #print('***stop index will be: ', index)
                    stop_index_list.append(index)
            count = 0
    
    return stop_index_list

In [210]:
###test the fuction can run or not
list_stop_index_each_year = get_stop_index_each_year_to_R0_R9(2017)

year:  2017


In [211]:
def calcu_fomx_tox_sum(year):
	df_v7_each_year = df_v7_years.get_group(year)
	list_stop_index_each_year = get_stop_index_each_year_to_R0_R9(year)

	add_final_stop_list = list_stop_index_each_year.copy()
	add_final_stop_list.append(len(df_v7_each_year))

	from_list = [0]
	to_list = []
	sum_list = []

	print('->x 0 + ... + x', list_stop_index_each_year[0], '=', df_v7_each_year.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
	#print(df_v7_each_year.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
	to_list.append(list_stop_index_each_year[0])
	sum_list.append(df_v7_each_year.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())

	for i in range(0, len(add_final_stop_list)-1):
	    #if i == len(add_final_stop_list)-1:
	    #    break
	    #print(i, add_final_stop_list[i] )
	    print('->x', add_final_stop_list[i]+1, '+ ... + x', add_final_stop_list[i+1], '=', df_v7_each_year.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
	    #print(df_v7_each_year.n_of_posters.iloc[ list_stop_index_each_year[i]+1 : list_stop_index_each_year[i+1]+1 ].sum())
	    from_list.append(add_final_stop_list[i]+1)
	    to_list.append(add_final_stop_list[i+1])
	    sum_list.append(df_v7_each_year.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
	    #print('->x 184', '+ ... + x', len(df_v7_each_year), '=', df_v7_each_year.n_of_posters.iloc[ list_stop_index_each_year[i+1]+1 : len(df_v7_each_year) ].sum())

	print(from_list, len(from_list))
	print(to_list, len(to_list))
	print(sum_list, len(sum_list))
	return from_list, to_list, sum_list , df_v7_each_year
#for year in df_v7_years.groups.keys():
#    per_y_from_list, per_y_to_list, per_y_sum_list = calcu_fomx_tox_sum(year)

In [212]:
def create_df_each_year(from_list, to_list, sum_list):
	from_x = from_list
	to_x = to_list
	divided_sum_group = sum_list

	movie_num_each_group_dict = {
	                'from_x': from_x,
	                'to_x': to_x,
	                'n_sum_group': divided_sum_group
	                }

	year_sum_df = pd.DataFrame(movie_num_each_group_dict)
	return year_sum_df

In [214]:
def giving_label_each_group_n_csv(year_sum_df, df_v7_each_year):
    df_v7_each_year['sort_poster_label'] = 'N/A'
    range_num = 0
    for from_i in range(0, len(year_sum_df['from_x'])):
        print('from_i:', from_i)
        if from_i == 4:
            df_v7_each_year['sort_poster_label'].iloc[ year_sum_df['from_x'].iloc[from_i] : len(df_v7_1997) ] = 5
            break
        df_v7_each_year['sort_poster_label'].iloc[ year_sum_df['from_x'].iloc[from_i] : year_sum_df['from_x'].iloc[from_i+1] ] = range_num
        range_num = range_num + 1
    "{0}_with_label_sort_poster_5cate.csv".format(year)
    df_v7_each_year.to_csv("{0}_with_label_sort_poster.csv".format(year))

In [215]:
for year in df_v7_years.groups.keys():
    per_y_from_list, per_y_to_list, per_y_sum_list, per_df_v7_each_year = calcu_fomx_tox_sum(year)
    per_year_sum_df = create_df_each_year(per_y_from_list, per_y_to_list, per_y_sum_list)
    giving_label_each_group_n_csv(per_year_sum_df, per_df_v7_each_year)
    print()

year:  1997
->x 0 + ... + x 60 = 236
->x 61 + ... + x 118 = 235
->x 119 + ... + x 147 = 238
->x 148 + ... + x 172 = 237


/Users/linyujing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/linyujing/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/linyujing/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name_

->x 173 + ... + x 190 = 235
[0, 61, 119, 148, 173] 5
[60, 118, 147, 172, 190] 5
[236, 235, 238, 237, 235] 5
from_i: 0
from_i: 1
from_i: 2
from_i: 3
from_i: 4

year:  1998
->x 0 + ... + x 50 = 228
->x 51 + ... + x 103 = 228
->x 104 + ... + x 144 = 254
->x 145 + ... + x 173 = 235
->x 174 + ... + x 184 = 201
[0, 51, 104, 145, 174] 5
[50, 103, 144, 173, 184] 5
[228, 228, 254, 235, 201] 5
from_i: 0
from_i: 1
from_i: 2
from_i: 3
from_i: 4

year:  1999
->x 0 + ... + x 41 = 205
->x 42 + ... + x 103 = 222
->x 104 + ... + x 152 = 208
->x 153 + ... + x 181 = 206
->x 182 + ... + x 197 = 203
[0, 42, 104, 153, 182] 5
[41, 103, 152, 181, 197] 5
[205, 222, 208, 206, 203] 5
from_i: 0
from_i: 1
from_i: 2
from_i: 3
from_i: 4

year:  2000
->x 0 + ... + x 56 = 165
->x 57 + ... + x 100 = 163
->x 101 + ... + x 141 = 169
->x 142 + ... + x 183 = 167
->x 184 + ... + x 205 = 156
[0, 57, 101, 142, 184] 5
[56, 100, 141, 183, 205] 5
[165, 163, 169, 167, 156] 5
from_i: 0
from_i: 1
from_i: 2
from_i: 3
from_i: 4

year

In [216]:
with open( '5_cate_sum_list_1997-2017.csv', 'w' , encoding='UTF-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        fileHeader = ['R0', 'R1', 'R2', 'R3' , 'R4']
        csv_writer.writerow(fileHeader)
        for year in df_v7_years.groups.keys():
            per_y_from_list, per_y_to_list, per_y_sum_list, per_df_v7_each_year = calcu_fomx_tox_sum(year)
            csv_writer.writerow(per_y_sum_list)

year:  1997
->x 0 + ... + x 60 = 236
->x 61 + ... + x 118 = 235
->x 119 + ... + x 147 = 238
->x 148 + ... + x 172 = 237
->x 173 + ... + x 190 = 235
[0, 61, 119, 148, 173] 5
[60, 118, 147, 172, 190] 5
[236, 235, 238, 237, 235] 5
year:  1998
->x 0 + ... + x 50 = 228
->x 51 + ... + x 103 = 228
->x 104 + ... + x 144 = 254
->x 145 + ... + x 173 = 235
->x 174 + ... + x 184 = 201
[0, 51, 104, 145, 174] 5
[50, 103, 144, 173, 184] 5
[228, 228, 254, 235, 201] 5
year:  1999
->x 0 + ... + x 41 = 205
->x 42 + ... + x 103 = 222
->x 104 + ... + x 152 = 208
->x 153 + ... + x 181 = 206
->x 182 + ... + x 197 = 203
[0, 42, 104, 153, 182] 5
[41, 103, 152, 181, 197] 5
[205, 222, 208, 206, 203] 5
year:  2000
->x 0 + ... + x 56 = 165
->x 57 + ... + x 100 = 163
->x 101 + ... + x 141 = 169
->x 142 + ... + x 183 = 167
->x 184 + ... + x 205 = 156
[0, 57, 101, 142, 184] 5
[56, 100, 141, 183, 205] 5
[165, 163, 169, 167, 156] 5
year:  2001
->x 0 + ... + x 62 = 230
->x 63 + ... + x 100 = 229
->x 101 + ... + x 149 = 

In [156]:
per_y_from_list, per_y_to_list, per_y_sum_list, df_v7_each_year = calcu_fomx_tox_sum(2000)
year_sum_df = create_df_each_year(per_y_from_list, per_y_to_list, per_y_sum_list)
year_sum_df

year:  2000
->x 0 + ... + x 31 = 77
->x 32 + ... + x 60 = 114
->x 61 + ... + x 88 = 85
->x 89 + ... + x 110 = 81
->x 111 + ... + x 132 = 101
->x 133 + ... + x 152 = 86
->x 153 + ... + x 174 = 82
->x 175 + ... + x 191 = 74
->x 192 + ... + x 204 = 120
->x 205 + ... + x 205 = 0
[0, 32, 61, 89, 111, 133, 153, 175, 192, 205] 10
[31, 60, 88, 110, 132, 152, 174, 191, 204, 205] 10
[77, 114, 85, 81, 101, 86, 82, 74, 120, 0] 10


,from_x,n_sum_group,to_x
0,0,77,31
1,32,114,60
2,61,85,88
3,89,81,110
4,111,101,132
5,133,86,152
6,153,82,174
7,175,74,191
8,192,120,204
9,205,0,205


In [157]:
year_sum_df.n_sum_group.sum()

820

In [97]:
#df_v7_1997['sort_poster_label'].iloc[ 0 : year_sum_df['from_x'].iloc[] ] = 9
range_num = 0
for from_i in range(0, len(year_sum_df['from_x'])):
    print('from_i:', from_i)
    if from_i == 9:
        df_v7_1997['sort_poster_label'].iloc[ year_sum_df['from_x'].iloc[from_i] : len(df_v7_1997) ] = 9
        break
    df_v7_1997['sort_poster_label'].iloc[ year_sum_df['from_x'].iloc[from_i] : year_sum_df['from_x'].iloc[from_i+1] ] = range_num
    range_num = range_num + 1
#df_v7_1997['sort_poster_label'].iloc[]

In [96]:
df_v7_1997

,n_index,previous_index_num,movie_title_txt_num,imdb_id,n_of_posters,total_box_office,with_adjusted_box_office,total_label,year,movie_title,year_box_office_label,Rnum_year_box_office_label,groups_num,train_or_test,budget,sort_poster_label
0,10037,5588,5854,tt0120376,2,217,422,0,1997,Trojan War,0,R0,18,test,0,0
1,10036,5576,5839,tt0118870,3,552,1076,0,1997,Cold Around the Heart,0,R0,17,test,0,0
2,10035,5548,5800,tt0130297,1,967,1606,0,1997,Sweet Jane,0,R0,16,test,0,0
3,10034,5467,5708,tt0028905,1,2126,3534,0,1997,Homme Nu L,0,R0,15,test,0,0
4,10033,5365,5592,tt0119327,2,3620,7068,0,1997,Hugo Pool,0,R0,14,train,0,0
5,10020,5259,5471,tt0119401,1,5000,9768,0,1997,James Gang The,0,R0,1,train,0,0
6,10021,5265,5477,tt0118811,5,5000,9768,0,1997,Call of the Wild Dog of the Yukon The,0,R0,2,train,0,0
7,10022,5269,5483,tt7253216,1,5000,9768,0,1997,Guy,0,R0,3,train,0,0
8,10023,5270,5484,tt0113646,2,5000,9768,0,1997,Lewis and Clark and George,0,R0,4,train,0,0
9,10024,5273,5487,tt0111453,3,5000,9768,0,1997,Tollbooth,0,R0,5,train,0,0


In [91]:
df_v7_1997.to_csv('1997_with_label_sort_poster.csv')

#

In [31]:
from_x = from_list
to_x = to_list
divided_sum_group = sum_list

movie_num_each_group_dict = {
                'from_x': from_x,
                'to_x': to_x,
                'n_sum_group': divided_sum_group
                }

year_sum_df = pd.DataFrame(movie_num_each_group_dict)
#year_sum_df.reindex(columns=['from_x', 'to_x', 'n_sum_group'])
year_sum_df

,from_x,n_sum_group,to_x
0,0,118,26
1,27,118,60
2,61,119,87
3,88,116,118
4,119,116,135
5,136,130,148
6,149,129,161
7,162,138,174
8,175,105,183
9,184,92,190


In [119]:
df_v7_2016 = df_v7_years.get_group(2016)
list_stop_index_each_year = get_stop_index_each_year_to_R0_R9(2016)

add_final_stop_list = list_stop_index_each_year.copy()
add_final_stop_list.append(len(df_v7_2016))

from_list = [0]
to_list = []
sum_list = []

print('->x 0 + ... + x', list_stop_index_each_year[0], '=', df_v7_2016.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
#print(df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
to_list.append(list_stop_index_each_year[0])
sum_list.append(df_v7_2016.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())

for i in range(0, len(add_final_stop_list)-1):
    #if i == len(add_final_stop_list)-1:
    #    break
    #print(i, add_final_stop_list[i] )
    print('->x', add_final_stop_list[i]+1, '+ ... + x', add_final_stop_list[i+1], '=', df_v7_2016.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print(df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i]+1 : list_stop_index_each_year[i+1]+1 ].sum())
    from_list.append(add_final_stop_list[i]+1)
    to_list.append(add_final_stop_list[i+1])
    sum_list.append(df_v7_2016.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print('->x 184', '+ ... + x', len(df_v7_1997), '=', df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i+1]+1 : len(df_v7_1997) ].sum())

print(from_list, len(from_list))
print(to_list, len(to_list))
print(sum_list, len(sum_list))

#[411, 733, 979, 1226, 1380, 1481, 1538, 1573, 1601]

year:  2016
->x 0 + ... + x 411 = 1815
->x 412 + ... + x 733 = 1809
->x 734 + ... + x 979 = 1813
->x 980 + ... + x 1226 = 1804
->x 1227 + ... + x 1380 = 1841
->x 1381 + ... + x 1481 = 1839
->x 1482 + ... + x 1538 = 1804
->x 1539 + ... + x 1573 = 1870
->x 1574 + ... + x 1601 = 1822
->x 1602 + ... + x 1616 = 1685
[0, 412, 734, 980, 1227, 1381, 1482, 1539, 1574, 1602] 10
[411, 733, 979, 1226, 1380, 1481, 1538, 1573, 1601, 1616] 10
[1815, 1809, 1813, 1804, 1841, 1839, 1804, 1870, 1822, 1685] 10


In [116]:
df_v7_2017 = df_v7_years.get_group(2017)

list_stop_index_each_year

add_final_stop_list = list_stop_index_each_year.copy()
add_final_stop_list.append(len(df_v7_2017))

from_list = [0]
to_list = []
sum_list = []

print('->x 0 + ... + x', list_stop_index_each_year[0], '=', df_v7_2017.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
#print(df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
to_list.append(list_stop_index_each_year[0])
sum_list.append(df_v7_2017.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())

for i in range(0, len(add_final_stop_list)-1):
    #if i == len(add_final_stop_list)-1:
    #    break
    #print(i, add_final_stop_list[i] )
    print('->x', add_final_stop_list[i]+1, '+ ... + x', add_final_stop_list[i+1], '=', df_v7_2017.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print(df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i]+1 : list_stop_index_each_year[i+1]+1 ].sum())
    from_list.append(add_final_stop_list[i]+1)
    to_list.append(add_final_stop_list[i+1])
    sum_list.append(df_v7_2017.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print('->x 184', '+ ... + x', len(df_v7_1997), '=', df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i+1]+1 : len(df_v7_1997) ].sum())

print(from_list, len(from_list))
print(to_list, len(to_list))
print(sum_list, len(sum_list))

#[260, 482, 689, 862, 990, 1082, 1120, 1143, 1163]

->x 0 + ... + x 260 = 1040
->x 261 + ... + x 482 = 1035
->x 483 + ... + x 689 = 1023
->x 690 + ... + x 862 = 1046
->x 863 + ... + x 990 = 1112
->x 991 + ... + x 1082 = 1071
->x 1083 + ... + x 1120 = 1053
->x 1121 + ... + x 1143 = 1029
->x 1144 + ... + x 1163 = 1085
->x 1164 + ... + x 1174 = 862
[0, 261, 483, 690, 863, 991, 1083, 1121, 1144, 1164] 10
[260, 482, 689, 862, 990, 1082, 1120, 1143, 1163, 1174] 10
[1040, 1035, 1023, 1046, 1112, 1071, 1053, 1029, 1085, 862] 10


In [99]:
df_v7_year.groups.keys()

dict_keys([1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [100]:
list_stop_index_all_year = []
for year in df_v7_years.groups.keys():
    list_stop_index_each_year = get_stop_index_each_year_to_R0_R9(year)
    list_stop_index_all_year.append(list_stop_index_each_year)

year:  1997
year:  1998
year:  1999
year:  2000
year:  2001
year:  2002
year:  2003
year:  2004
year:  2005
year:  2006
year:  2007
year:  2008
year:  2009
year:  2010
year:  2011
year:  2012
year:  2013
year:  2014
year:  2015
year:  2016
year:  2017


###

In [144]:
list_stop_index_each_year

[26, 60, 87, 118, 135, 148, 161, 174, 183]

In [15]:
add_final_stop_list = list_stop_index_each_year.copy()

In [16]:
add_final_stop_list

[26, 60, 87, 118, 135, 148, 161, 174, 183]

In [17]:
add_final_stop_list.append(len(df_v7_1997))
add_final_stop_list

[26, 60, 87, 118, 135, 148, 161, 174, 183, 190]

In [27]:
from_list = [0]
to_list = []
sum_list = []

print('->x 0 + ... + x', list_stop_index_each_year[0], '=', df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
#print(df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
to_list.append(list_stop_index_each_year[0])
sum_list.append(df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())

for i in range(0, len(add_final_stop_list)-1):
    #if i == len(add_final_stop_list)-1:
    #    break
    #print(i, add_final_stop_list[i] )
    print('->x', add_final_stop_list[i]+1, '+ ... + x', add_final_stop_list[i+1], '=', df_v7_1997.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print(df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i]+1 : list_stop_index_each_year[i+1]+1 ].sum())
    from_list.append(add_final_stop_list[i]+1)
    to_list.append(add_final_stop_list[i+1])
    sum_list.append(df_v7_1997.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print('->x 184', '+ ... + x', len(df_v7_1997), '=', df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i+1]+1 : len(df_v7_1997) ].sum())

print(from_list, len(from_list))
print(to_list, len(to_list))
print(sum_list, len(sum_list))

->x 0 + ... + x 26 = 118
->x 27 + ... + x 60 = 118
->x 61 + ... + x 87 = 119
->x 88 + ... + x 118 = 116
->x 119 + ... + x 135 = 116
->x 136 + ... + x 148 = 130
->x 149 + ... + x 161 = 129
->x 162 + ... + x 174 = 138
->x 175 + ... + x 183 = 105
->x 184 + ... + x 190 = 92
[0, 27, 61, 88, 119, 136, 149, 162, 175, 184] 10
[26, 60, 87, 118, 135, 148, 161, 174, 183, 190] 10
[118, 118, 119, 116, 116, 130, 129, 138, 105, 92] 10


###

In [36]:
######now foucse


for range_num in range(0, len(year_sum_df)):
    n_sum_poster = year_sum_df.n_sum_group.iloc[range_num]
    print(n_sum_poster)
    train_test_thold = int(n_sum_poster*0.8)
    print(train_test_thold)
    
    
    
    
    
def divide_train_test_each_group(group_nu): #year
    stop_index_list = []
    #prepared inital paraments
    count = 0
    df_v7_each_year = df_v7_years.get_group(year)
    
    print('year: ', year)
    n_poster_num = df_v7_each_year['n_of_posters'].sum()
    print('number of poster: ', n_poster_num)
    group_num_threadhold = int(n_poster_num/10)
    print('group num threadhold: ', group_num_threadhold)
    for index in range(0, len(df_v7_each_year)):
        #print('index: ', index)
        n_poster_each_index = df_v7_each_year['n_of_posters'].iloc[index]
        #print('number of posters in this index: ', n_poster_each_index)
        count = count + n_poster_each_index
        if count >= group_num_threadhold: 
            print('***stop index: ', index)
            if count == group_num_threadhold:
                stop_index_list.append(index)
            #print('count: ', count)
            else:
                #print('nooooo')
                prev_count = count-df_v7_each_year['n_of_posters'].iloc[index]
                #print('previous: ', prev_count)
                #print('over: ', count)
                distant_prev = group_num_threadhold - prev_count
                #print('distant of prev_count: ', distant_prev)
                distant_over = count - group_num_threadhold
                #print('distant of over_count: ', distant_over)
                find_min = [distant_prev, distant_over]
                min_dis = min(find_min)
                #print('who is min: ', min_dis)
                #choose the corresponding index of the minimum number
                choose_index = find_min.index(min_dis)
                #print('choose index: ', choose_index)
                #0=prev, 1=over
                if choose_index == 0:
                    #print('I choose previous: ', prev_count)
                    print('***stop index will be: ', index-1)
                    stop_index_list.append(index-1)
                else:
                    #print('I choose over: ', count)
                    print('***stop index will be: ', index)
                    stop_index_list.append(index)
            count = 0
    return stop_index_list

118
94

118
94

119
95

116
92

116
92

130
104

129
103

138
110

105
84

92
73



In [ ]:
df_v7_1997.n_of_posters.sum()

In [ ]:
df_v7_1997.n_of_posters[0]

In [ ]:
df_v7_years.get_group(1997)['n_of_posters'].sum()

In [ ]:
df_v7_years.groups.keys()
list_year = list(df_v7_years.groups.keys())
#list_year

In [ ]:
list_stop_index_each_year

In [ ]:
list_stop_index_each_year.append(len(df_v7_1997))

In [ ]:
list_stop_index_each_year

In [ ]:
sum_list

In [ ]:
train_test_num_threadhold = int(group_num_threadhold*0.8)

def divide_train_tes_get_stop_index_each_year_to_R0_R9(year):
    for 
    stop_index_list = []
    #prepared inital paraments
    count = 0
    df_v7_each_year = df_v7_years.get_group(year)
    
    print('year: ', year)
    n_poster_num = df_v7_each_year['n_of_posters'].sum()
    print('number of poster: ', n_poster_num)
    group_num_threadhold = int(n_poster_num/10)
    print('group num threadhold: ', group_num_threadhold)
    train_test_num_threadhold = int(group_num_threadhold*0.8)
    for index in range(0, len(df_v7_each_year)):
        #print('index: ', index)
        n_poster_each_index = df_v7_each_year['n_of_posters'].iloc[index]
        #print('number of posters in this index: ', n_poster_each_index)
        count = count + n_poster_each_index
        if count >= group_num_threadhold: 
            #print('***stop index: ', index)
            if count == group_num_threadhold:
                stop_index_list.append(index)
                print('***stop index: ', index)
            #print('count: ', count)
            else:
                #print('nooooo')
                prev_count = count-df_v7_each_year['n_of_posters'].iloc[index]
                #print('previous: ', prev_count)
                #print('over: ', count)
                distant_prev = group_num_threadhold - prev_count
                #print('distant of prev_count: ', distant_prev)
                distant_over = count - group_num_threadhold
                #print('distant of over_count: ', distant_over)
                find_min = [distant_prev, distant_over]
                min_dis = min(find_min)
                #print('who is min: ', min_dis)
                #choose the corresponding index of the minimum number
                choose_index = find_min.index(min_dis)
                #print('choose index: ', choose_index)
                #0=prev, 1=over
                if choose_index == 0:
                    #print('I choose previous: ', prev_count)
                    print('***stop index will be: ', index-1)
                    stop_index_list.append(index-1)
                else:
                    #print('I choose over: ', count)
                    print('***stop index will be: ', index)
                    stop_index_list.append(index)
            count = 0
    return stop_index_list

In [ ]:
divide_train_tes_get_stop_index_each_year_to_R0_R9(1997)

####

In [56]:
df_v7_1997.n_of_posters.iloc[0:27].sum()

118

In [61]:
df_v7_1997.n_of_posters.iloc[27:61].sum()

118

In [62]:
df_v7_1997.n_of_posters.iloc[61:88].sum()

119

In [63]:
df_v7_1997.n_of_posters.iloc[88:119].sum()

116

In [64]:
df_v7_1997.n_of_posters.iloc[119:136].sum()

116

In [65]:
df_v7_1997.n_of_posters.iloc[136:149].sum()

130

In [66]:
df_v7_1997.n_of_posters.iloc[149:162].sum()

129

In [67]:
df_v7_1997.n_of_posters.iloc[162:175].sum()

138

In [68]:
df_v7_1997.n_of_posters.iloc[175:184].sum()

105

In [69]:
df_v7_1997.n_of_posters.iloc[184:190].sum()

92

#

In [93]:
y_1997_csv = pd.read_csv('1997_with_label_sort_poster.csv')

In [94]:
y_1997_csv

,Unnamed: 0,n_index,previous_index_num,movie_title_txt_num,imdb_id,n_of_posters,total_box_office,with_adjusted_box_office,total_label,year,movie_title,year_box_office_label,Rnum_year_box_office_label,groups_num,train_or_test,budget,sort_poster_label
0,0,10037,5588,5854,tt0120376,2,217,422,0,1997,Trojan War,0,R0,18,test,0,0
1,1,10036,5576,5839,tt0118870,3,552,1076,0,1997,Cold Around the Heart,0,R0,17,test,0,0
2,2,10035,5548,5800,tt0130297,1,967,1606,0,1997,Sweet Jane,0,R0,16,test,0,0
3,3,10034,5467,5708,tt0028905,1,2126,3534,0,1997,Homme Nu L,0,R0,15,test,0,0
4,4,10033,5365,5592,tt0119327,2,3620,7068,0,1997,Hugo Pool,0,R0,14,train,0,0
5,5,10020,5259,5471,tt0119401,1,5000,9768,0,1997,James Gang The,0,R0,1,train,0,0
6,6,10021,5265,5477,tt0118811,5,5000,9768,0,1997,Call of the Wild Dog of the Yukon The,0,R0,2,train,0,0
7,7,10022,5269,5483,tt7253216,1,5000,9768,0,1997,Guy,0,R0,3,train,0,0
8,8,10023,5270,5484,tt0113646,2,5000,9768,0,1997,Lewis and Clark and George,0,R0,4,train,0,0
9,9,10024,5273,5487,tt0111453,3,5000,9768,0,1997,Tollbooth,0,R0,5,train,0,0


####

In [ ]:
#####backup

sum_list = []
print('->x 0 + ... + x', list_stop_index_each_year[0], '=', df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
#print(df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
sum_list.append(df_v7_1997.n_of_posters.iloc[ 0 : list_stop_index_each_year[0]+1 ].sum())
for i in range(0, len(add_final_stop_list)-1):
    #if i == len(add_final_stop_list)-1:
    #    break
    print(i, add_final_stop_list[i] )
    print('->x', add_final_stop_list[i]+1, '+ ... + x', add_final_stop_list[i+1], '=', df_v7_1997.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print(df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i]+1 : list_stop_index_each_year[i+1]+1 ].sum())
    sum_list.append(df_v7_1997.n_of_posters.iloc[ add_final_stop_list[i]+1 : add_final_stop_list[i+1]+1 ].sum())
    #print('->x 184', '+ ... + x', len(df_v7_1997), '=', df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i+1]+1 : len(df_v7_1997) ].sum())

#sum_list.append(df_v7_1997.n_of_posters.iloc[ list_stop_index_each_year[i+1]+1 : len(df_v7_1997) ].sum())

In [ ]:

#divide each year into indiviual group
df_v7_years = df_v7.groupby('year')

for year in list_year:
    print(year)
    print(df_v7_years.get_group(year)['n_of_posters'].sum())  

In [ ]:
count = 0

for i in range(0, len(df_v7_1997.n_of_posters)):
    print('i: ', i)
    print('number of posters:', df_v7_1997.n_of_posters[i])
    count = count + df_v7_1997.n_of_posters[i]
    print('count: ', count)  
    train_test_theadhold = int(int(int(df_v7_1997.n_of_posters.sum()*0.8)*0.8)/10)
    print('train/test theadhold: ', train_test_theadhold)
    train_vali_theadhold = int((int(df_v7_1997.n_of_posters.sum()*0.8) - int(int(df_v7_1997.n_of_posters.sum()*0.8)*0.8))/10)
    print('train/vali theadhold: ',train_vali_theadhold)
    if count > int(int(int(df_v7_1997.n_of_posters.sum()*0.8)*0.8)/10): ##training data
        print('dadadadadadadadada')
        print('imdb:', df_v7.imdb_id[i])
        print('n_of_posters: ',df_v7_1997.n_of_posters[i])
        print('now count: ', count)
        prev_count = count-df_v7_1997.n_of_posters[i]
        print('previous count: ', prev_count) 
        if prev_count-train_test_theadhold
        #df_v7_2017.sort_b_off_tra_va_test[i]
        
    print()
        
count

In [ ]:
df_v7_2017.iloc[1]

In [ ]:
df_v7_2017.loc[1]